# 05 Two-way (column or row) relative frequency tables

In [96]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go

In [97]:
import findspark; findspark.init()
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql import functions as F
spark = SparkSession.builder.appName('statistics').master('local').getOrCreate()

[khanacademy](https://www.khanacademy.org/math/ap-statistics/analyzing-categorical-ap/stats-two-way-tables/v/two-way-relative-frequency-tables?modal=1)


![Two-way (column or row) relative frequency tables fig 1](./imgs/01-05-01.png)

In [98]:
dataset = {
    'Car': 28*['SUV'] + 35*['Sport car']  + 97*['SUV'] + 104*['Sport car'] ,
    'Accident': 28*['yes'] + 35*['yes'] + 97*['no'] + 104*['no']
}

In [99]:
df = pd.DataFrame(dataset)
df

Car Accident
0          SUV      yes
1          SUV      yes
2          SUV      yes
3          SUV      yes
4          SUV      yes
..         ...      ...
259  Sport car       no
260  Sport car       no
261  Sport car       no
262  Sport car       no
263  Sport car       no

[264 rows x 2 columns]

In [100]:
sdf = spark.createDataFrame(zip(*dataset.values()), schema=list(dataset.keys()))
sdf.registerTempTable('sdf_table')
sdf.show()

+---+--------+
|Car|Accident|
+---+--------+
|SUV|     yes|
|SUV|     yes|
|SUV|     yes|
|SUV|     yes|
|SUV|     yes|
|SUV|     yes|
|SUV|     yes|
|SUV|     yes|
|SUV|     yes|
|SUV|     yes|
|SUV|     yes|
|SUV|     yes|
|SUV|     yes|
|SUV|     yes|
|SUV|     yes|
|SUV|     yes|
|SUV|     yes|
|SUV|     yes|
|SUV|     yes|
|SUV|     yes|
+---+--------+
only showing top 20 rows



In [101]:
df.groupby(['Accident', 'Car']).size()

Accident  Car      
no        SUV           97
          Sport car    104
yes       SUV           28
          Sport car     35
dtype: int64

In [102]:
sdf.groupby('Accident', 'Car').count().show()

+--------+---------+-----+
|Accident|      Car|count|
+--------+---------+-----+
|      no|      SUV|   97|
|     yes|Sport car|   35|
|      no|Sport car|  104|
|     yes|      SUV|   28|
+--------+---------+-----+



In [103]:
spark.sql('select Accident, Car, count(*) as count from sdf_table group by Accident, Car').show()

+--------+---------+-----+
|Accident|      Car|count|
+--------+---------+-----+
|      no|      SUV|   97|
|     yes|Sport car|   35|
|      no|Sport car|  104|
|     yes|      SUV|   28|
+--------+---------+-----+



In [104]:
two_way_table = pd.crosstab(df['Accident'], df['Car'])
two_way_table

Car       SUV  Sport car
Accident                
no         97        104
yes        28         35

In [105]:
s_two_way_table = sdf.crosstab('Accident', 'Car')
s_two_way_table.show()

+------------+---+---------+
|Accident_Car|SUV|Sport car|
+------------+---+---------+
|         yes| 28|       35|
|          no| 97|      104|
+------------+---+---------+



In [106]:
freq_table = two_way_table.copy()
freq_table['SUV'] = two_way_table['SUV'] / two_way_table['SUV'].sum()
freq_table['Sport car'] = two_way_table['Sport car'] / two_way_table['Sport car'].sum()
freq_table

Car         SUV  Sport car
Accident                  
no        0.776   0.748201
yes       0.224   0.251799

In [107]:
s_freq_table = s_two_way_table
s_freq_table = s_freq_table.withColumn('SUV', F.col('SUV') / s_freq_table.select(F.sum('SUV')).collect()[0][0])
s_freq_table = s_freq_table.withColumn('Sport car', F.col('Sport car') / s_freq_table.select(F.sum('Sport car')).collect()[0][0])
s_freq_table.show()

+------------+-----+------------------+
|Accident_Car|  SUV|         Sport car|
+------------+-----+------------------+
|         yes|0.224|0.2517985611510791|
|          no|0.776|0.7482014388489209|
+------------+-----+------------------+



In [108]:
freq_table = freq_table.append(freq_table.sum().rename('Total'))
freq_table

Car         SUV  Sport car
Accident                  
no        0.776   0.748201
yes       0.224   0.251799
Total     1.000   1.000000